In [10]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine
import os

def get_exchange_usd_rates() -> pd.DataFrame:
    today = datetime.now().date()
    week_ago = today - timedelta(days=7)

    exchange_rates = []

    for date in pd.date_range(week_ago, today):
        url = f'https://www.cbr-xml-daily.ru/archive/{date:%Y/%m/%d}/daily_json.js'
        res = requests.get(url)
        if res.status_code == 200:
            data = res.json()
            usd_data = data['Valute'].get('USD')
            if usd_data:
                exchange_rates.append({
                    'date': date,
                    'rate': usd_data['Value'],
                    'nominal': usd_data['Nominal'],
                    'desc': 'Доллар США'
                })

    return pd.DataFrame(exchange_rates)

In [11]:
def load_rates_into_db(df: pd.DataFrame):
    # Получаем конфигурацию для подключения к БД из переменных окружения
    pg_host = os.getenv('PG_HOST', 'localhost')
    pg_port = os.getenv('PG_PORT', '5432')
    pg_user = os.getenv('PG_USER', 'postgres')
    pg_password = os.getenv('PG_PASSWORD', 'root')
    pg_database = os.getenv('PG_DATABASE', 'dwh_dbt')

    try:
        # Создаем соединение с базой данных
        with create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}').connect() as conn:
            # Сохраняем датафрейм в базу данных
            df.to_sql('usd_rates', conn, if_exists='replace', index=False)
            print("Данные успешно загружены в базу данных.")
    except Exception as e:
        print(f"Произошла ошибка при загрузке данных в базу данных: {e}")


load_rates_into_db(get_exchange_usd_rates())
